# Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import random
import pandas as pd
import os
import time
import copy
import librosa
import librosa.display
import IPython
from IPython.display import Audio
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_color_map
from PIL import Image
from random import randint
import seaborn as sns
import sys

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.GELU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.gelu = nn.GELU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.gelu(out)
        return out


In [ ]:
class ParallelModel(nn.Module):
    def __init__(self, block, layers, num_emotions):
        super().__init__()

        # Resnet Block
        self.inplanes = 64
        self.res_conv1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size = 3, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.GELU())
        self.res_maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.res_layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.res_layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.res_layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.res_layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.res_avgpool = nn.AvgPool2d(3, stride=1)
#         self.res_fc = nn.Linear(512, num_emotions)

        # Block 1:
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=16,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.3)

        # Block 2:
        self.conv2 = nn.Conv2d(in_channels=16,
                               out_channels=32,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.max_pool2 = nn.MaxPool2d(kernel_size=4, stride=4)

        # Block 3:
        self.conv3 = nn.Conv2d(in_channels=32,
                               out_channels=64,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.max_pool3 = nn.MaxPool2d(kernel_size=4, stride=4)

        # Block 4:
        self.conv4 = nn.Conv2d(in_channels=64,
                               out_channels=64,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.max_pool4 = nn.MaxPool2d(kernel_size=4, stride=4)

        # LSTM block
        self.lstm_maxpool = nn.MaxPool2d(kernel_size=[2,4], stride=[2,4])
        hidden_size = 148
        self.lstm = nn.LSTM(input_size=74,hidden_size=hidden_size,bidirectional=True, batch_first=True)
        self.dropout_lstm = nn.Dropout(0.1)
        self.attention_linear = nn.Linear(2*hidden_size,1) # 2*hidden_size for the 2 outputs of bidir LSTM

        # Transformer block
        self.transf_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
        transf_layer = nn.TransformerEncoderLayer(d_model=148, nhead=4, dim_feedforward=512, dropout=0.4, activation='relu')
        self.transf_encoder = nn.TransformerEncoder(transf_layer, num_layers=4)
        # Linear softmax layer
        self.out_linear = nn.Linear(2*hidden_size+212+15360,num_emotions)
        self.dropout_linear = nn.Dropout(p=0)
        self.out_softmax = nn.Softmax(dim=1)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self,x):

        # resnet embedding
        res_embedding = self.res_conv1(x)
        res_embedding = self.res_maxpool(res_embedding)
        res_embedding = self.res_layer0(res_embedding)
        res_embedding = self.res_layer1(res_embedding)
        res_embedding = self.dropout(res_embedding)
        res_embedding = self.res_layer2(res_embedding)
        res_embedding = self.res_layer3(res_embedding)
        res_embedding = self.dropout(res_embedding)

        # res_embedding = self.res_avgpool(res_embedding)
        res_embedding = res_embedding.view(res_embedding.size(0), -1)

        # conv embedding
#         conv_embedding = self.conv2Dblock(x) #(b,channel,freq,time)
        conv_embedding = self.dropout(self.max_pool1(self.relu(self.bn1(self.conv1(x)))))
        conv_embedding = self.dropout(self.max_pool2(self.relu(self.bn2(self.conv2(conv_embedding)))))
        conv_embedding = self.dropout(self.max_pool3(self.relu(self.bn3(self.conv3(conv_embedding)))))
        conv_embedding = self.dropout(self.max_pool4(self.relu(self.bn4(self.conv4(conv_embedding)))))

        conv_embedding = torch.flatten(conv_embedding, start_dim=1) # do not flatten batch dimension

        # lstm embedding
        x_reduced = self.lstm_maxpool(x)
        x_reduced = torch.squeeze(x_reduced,1)
        x_reduced = x_reduced.permute(0,2,1) # (batch,time,freq)
        lstm_embedding, (h,c) = self.lstm(x_reduced) # (b, time, hidden_size*2)
        lstm_embedding = self.dropout_lstm(lstm_embedding)
        batch_size,T,_ = lstm_embedding.shape
        attention_weights = [None]*T
        for t in range(T):
            embedding = lstm_embedding[:,t,:]
            attention_weights[t] = self.attention_linear(embedding)
        attention_weights_norm = nn.functional.softmax(torch.stack(attention_weights,-1),-1)
        attention = torch.bmm(attention_weights_norm,lstm_embedding) # (Bx1xT)*(B,T,hidden_size*2)=(B,1,2*hidden_size)
        attention = torch.squeeze(attention, 1)


        # transformer embedding
        x_reduced_t = self.transf_maxpool(x)
        x_reduced_t = torch.squeeze(x_reduced_t,1)
        x_reduced_t = x_reduced_t.permute(2,0,1) # (time,batch,embedding)
        transf_out = self.transf_encoder(x_reduced_t)
        transf_embedding = torch.mean(transf_out, dim=0)
        # concatenate
        complete_embedding = torch.cat([res_embedding, conv_embedding, attention, transf_embedding], dim=1)
        # final Linear
        output_logits = self.out_linear(complete_embedding)
        output_logits = self.dropout_linear(output_logits)
        output_softmax = self.out_softmax(output_logits)
        return output_logits, output_softmax


In [ ]:
EMOTIONS = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 0:'surprise'}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Selected device is {}'.format(device))

show_model = ParallelModel(ResidualBlock, [2,2,2,2], len(EMOTIONS))
show_model.to(device)
# print(show_model)


X_train = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/xtrain.npy")
X_test = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/xtest.npy")
X_val = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/xval.npy")

Y_train = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/ytrain.npy")
Y_test = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/ytest.npy")
Y_val = np.load(file="/content/drive/MyDrive/serdl/notebooks/3_dataset_mel/yval.npy")

print(X_train.shape)

# Load model

In [ ]:
LOAD_PATH = os.path.join(os.getcwd(),'/content/drive/MyDrive/serdl/notebooks/new_models')
model = ParallelModel(ResidualBlock, [2,2,2,2], len(EMOTIONS))
model.load_state_dict(torch.load(os.path.join(LOAD_PATH,'big_ctrl_mel_model.pt')))
print('Model is loaded from {}'.format(os.path.join(LOAD_PATH,'big_ctrl_mel_model.pt')))

# Feature Activation Visualization (only for CNN-based models)

In [ ]:
def apply_colormap_on_image(org_im, activation, colormap_name):
    """
        Apply heatmap on image
    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): Activation map (grayscale) 0-255
        colormap_name (str): Name of the colormap
    """
    # Get colormap
    color_map = mpl_color_map.get_cmap(colormap_name)
    no_trans_heatmap = color_map(activation)
    # Change alpha channel in colormap to make sure original image is displayed
    heatmap = copy.copy(no_trans_heatmap)
    heatmap[:, :, 3] = 0.4
    heatmap = Image.fromarray((heatmap*255).astype(np.uint8))
    no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

    # Apply heatmap on iamge
    heatmap_on_image = Image.new("RGBA", org_im.size)
    heatmap_on_image = Image.alpha_composite(heatmap_on_image, org_im.convert('RGBA'))
    heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap)
    return no_trans_heatmap, heatmap_on_image

In [ ]:
gradients = []  # A gloabl variable used to save the gradient
def generate_afm(model, input_image):
    
    def forward_hook(module, f_input, f_output):
        """
        A function to get feature maps in forward calculation

        Inputs
        - module: non-modified system variable
        - f_input: feature map generated by last conv layer
        - f_output: feature map generated by this conv layer

        Append feature maps generated by this conv layer to a list each time
        """
        feature_map_block.append(f_output)

    def backward_hook(module, grad_in, grad_out):
        """
        A function to get gradients in back-propagation

        Inputs
        - module: non-modified system variable
        - grad_in: gradient passed in from last fc layer
        - grad_output: gradient passed out from this conv layer

        Append gradients generated by this conv layer to a list each time
        """
        gradient_block.append(grad_out[0].detach())

    def get_class_loss(out_vec, index=None):
        if not index:
            index = np.argmax(out_vec.cpu().data.numpy())
        else:
            index = np.array(index)
        index = index[np.newaxis, np.newaxis]
        index = torch.from_numpy(index)
        one_hot = torch.zeros(1, 8).scatter_(1, index, 1)
        one_hot.requires_grad = True
        one_hot = one_hot.to(device)
        class_loss = torch.sum(one_hot * out_vec)

        return class_loss

    feature_map_block = list() # feature map
    gradient_block = list()

    model.train()
    # register hook
    model.conv4.register_forward_hook(forward_hook)
    model.conv4.register_backward_hook(backward_hook)

    # forward
    output_logits, output_softmax = model(input_image)

    # backward
    model.zero_grad()
    loss = get_class_loss(output_logits)
    loss.backward()

    # generate afm
    gradient = gradient_block[0].cpu().data.numpy().squeeze()
    fmap = feature_map_block[0].cpu().data.numpy().squeeze()
    afm = np.zeros(fmap.shape[1:], dtype=np.float32)
    weights = np.mean(gradient, axis=(1, 2))

    for i, w in enumerate(weights):
        afm += w * fmap[i, :, :]

    # Post processing
    afm = np.maximum(afm, 0)
    afm = (afm - np.min(afm)) / (np.max(afm) - np.min(afm))  # Normalize between 0-1
    afm = np.uint8(afm * 255)  # Scale between 0-255 to visualize
    afm = np.uint8(Image.fromarray(afm).resize((input_image.shape[2],
                    input_image.shape[3]), Image.ANTIALIAS))

    return afm

In [ ]:
start = time.time()

In [ ]:
emotion = 0
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_0.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum0.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm0.npy", arr=avgafm)

In [ ]:
emotion = 1
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_1.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum1.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm1.npy", arr=avgafm)

In [ ]:
emotion = 2
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_2.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum2.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm2.npy", arr=avgafm)

In [ ]:
emotion = 3
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_3.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum3.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm3.npy", arr=avgafm)

In [ ]:
emotion = 4
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_4.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum4.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm4.npy", arr=avgafm)

In [ ]:
emotion = 5
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_5.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum5.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm5.npy", arr=avgafm)

In [ ]:
emotion = 6
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_6.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum6.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm6.npy", arr=avgafm)

In [ ]:
emotion = 7
index = np.where(Y_train == emotion)
index = index[0]

heatmap_tensor_sum1 = torch.zeros(4, 148, 188)
heatmap_tensor_sum1.to(device)
model.to(device)

AFM = torch.zeros(188, 148)
AFM.to(device)
for i in index:
    im = X_train[i, :, :, :]
    im = np.expand_dims(im, axis=0)
    im_tensor = torch.tensor(im, device=device).float()

    org_im = np.squeeze(im)
    org_im_expand = np.expand_dims(org_im, axis=0)

    org_im_tensor = torch.tensor(org_im_expand, device=device).float()
    org_im_tensor = inverse_transform(org_im_tensor) # get PIL Image

    afm = generate_afm(model, im_tensor)
    AFM += afm;
    heatmap, heatmap_on_image = apply_colormap_on_image(org_im_tensor, afm.T, 'hsv')

    heatmap_img_tensor = transform(heatmap_on_image)
    heatmap_tensor_sum1 += heatmap_img_tensor

heatmap_tensor_sum1 = heatmap_tensor_sum1 / len(index)

avgafm = AFM / len(index)

heatmap_sum_img1 = inverse_transform(heatmap_tensor_sum1)
plt.imshow(heatmap_sum_img1)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_emotion_7.png")
plt.show()

np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum7.npy", arr=heatmap_tensor_sum1)
np.save(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm7.npy", arr=avgafm)

In [ ]:
heatmap_tensor_sum1 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum1.npy")
heatmap_tensor_sum2 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum2.npy")
heatmap_tensor_sum3 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum3.npy")
heatmap_tensor_sum4 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum4.npy")
heatmap_tensor_sum5 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum5.npy")
heatmap_tensor_sum6 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum6.npy")
heatmap_tensor_sum7 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum7.npy")
heatmap_tensor_sum0 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/heatmap_tensor_sum0.npy")

heatmap_tensor_sum = (heatmap_tensor_sum1
                     + heatmap_tensor_sum2
                     + heatmap_tensor_sum3
                     + heatmap_tensor_sum4
                     + heatmap_tensor_sum5
                     + heatmap_tensor_sum6
                     + heatmap_tensor_sum7
                     + heatmap_tensor_sum0) / 8

heatmap_tensor_sum = torch.tensor(heatmap_tensor_sum, device=device)

heatmap_sum_img = inverse_transform(heatmap_tensor_sum)
plt.imshow(heatmap_sum_img)
plt.savefig("/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_results/Visualization_AVG_for_ALL.png")
plt.show()

In [ ]:
avgafm0 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm0.npy")
avgafm1 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm1.npy")
avgafm2 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm2.npy")
avgafm3 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm3.npy")
avgafm4 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm4.npy")
avgafm5 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm5.npy")
avgafm6 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm6.npy")
avgafm7 = np.load(file="/content/drive/MyDrive/serdl/notebooks/Training_in_3_datasets/big_ctrl_mel_temp/avgafm7.npy")

afm_sum_avg = (avgafm0
         + avgafm1
         + avgafm2
         + avgafm3
         + avgafm4
         + avgafm5
         + avgafm6
         + avgafm7) / 8

print(afm_sum_avg)

In [ ]:
fam = afm_sum_avg.T

Conv = np.zeros([44, 80])
for i in range(44):
    for j in range(80):
        Conv[i, j] = sum(sum(fam[i:i+104, j:j+108]))

print(Conv)

In [ ]:
index_max_conv = np.argmax(Conv)
max_conv = np.max(Conv)
print(index_max_conv)
print(max_conv)

In [ ]:
# index_min_conv = np.argmin(Conv)
# min_conv = np.min(Conv)
# print(index_min_conv)
# print(min_conv)

In [ ]:
row = index_max_conv // 80
col = -1 * (row * 80 - index_max_conv)

print(row, col)

In [ ]:
# row1 = index_min_conv // 80
# col1 = -1 * (row1 * 80 - index_min_conv)

# print(row1, col1)

In [ ]:
Conv[row, col]

In [ ]:
# Conv[row1, col1]